In [ ]:
import pandas as pd
import ast

file_path = 'final_data_keywords_flags.csv'  # Adjust the path as necessary
data = pd.read_csv(file_path)

data['Institutions'] = data['Institutions'].apply(
    lambda x: set(ast.literal_eval(x)) if pd.notnull(x) else set()
)

prompts = []
for institutions in data['Institutions']:
    for institution in institutions:
        prompt = f"Classify the following institution: {institution}. "
        prompt += "Is it an academic institution, company, or non-profit? "
        prompt += "Is it in the US or non-US, or is it unknown? "
        prompt += "If it is Non-US, what is the country? If in the USA, what state is it in? "
        prompt += "Within the USA, is it considered R1, R2, or neither, or is this unknown?"
        prompts.append(prompt)

print(prompts[0])


Classify the following institution: Prairie View A&M University. Is it an academic institution, company, or non-profit? Is it in the US or non-US, or is it unknown? If it is Non-US, what is the country? If in the USA, what state is it in? Within the USA, is it considered R1, R2, or neither, or is this unknown?


In [ ]:
import pandas as pd
import ast

file_path = 'final_data_keywords_flags.csv'
data = pd.read_csv(file_path)

def generate_prompt(row):
    institutions = row['Institutions']
    institutions_list = set([inst.strip(" '[]") for inst in institutions.split(',')])
    prompt = "For the following institution(s), provide the classification, location (US or non-US), the state (if in the US) or country (if outside the US), and research classification (R1, R2, or neither). The response should be in a structured format suitable for conversion into a dictionary.\n\nInstitution(s):\n"
    for institution in institutions_list:
        prompt += f"- {institution}\n"

    prompt += "\nPlease format the response as a JSON object, containing the fields 'institution', 'classification', 'location', 'country_or_state', and 'research_classification'. For example:\n\n{\n  \"institution\": \"[Name of the Institution]\",\n  \"classification\": \"[academic/non-academic]\",\n  \"location\": \"[US/non-US]\",\n  \"country_or_state\": \"[if US, state name; if non-US, country name]\",\n  \"research_classification\": \"[R1/R2/neither/don't know]\"\n}"

    return prompt

data['Prompt'] = data.apply(generate_prompt, axis=1)

data.to_csv('prompts.csv', index=False)

print(data['Prompt'][10])


For the following institution(s), provide the classification, location (US or non-US), the state (if in the US) or country (if outside the US), and research classification (R1, R2, or neither). The response should be in a structured format suitable for conversion into a dictionary.

Institution(s):
- Albert Einstein College of Medicine
- Beth Israel Deaconess Medical Center

Please format the response as a JSON object, containing the fields 'institution', 'classification', 'location', 'country_or_state', and 'research_classification'. For example:

{
  "institution": "[Name of the Institution]",
  "classification": "[academic/non-academic]",
  "location": "[US/non-US]",
  "country_or_state": "[if US, state name; if non-US, country name]",
  "research_classification": "[R1/R2/neither/don't know]"
}


In [ ]:
import openai
import pandas as pd

openai.api_key = ''
model = "gpt-3.5-turbo-0125"


prompts_data = pd.read_csv('prompts.csv')


results = []
import openai
import pandas as pd

prompts_data = pd.read_csv('prompts.csv')

results = []
for index, row in prompts_data.iterrows():
    prompt = row['Prompt']
    messages = [{"role": "system", "content": "Extract institutions information in JSON format. Follow the format in the prompt strictly"},
                {"role": "user", "content": prompt }]
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        extracted_info = response.choices[0].message.content
        results.append(extracted_info)

        if index % 300 == 0:
            responses_df = prompts_data.iloc[:index + 1]
            responses_df['Response'] = results


            responses_df.to_csv(f'responses_{index + 1}.csv', index=False)
    except openai.OpenAIError as e:
        print(f"An error occurred: {e}")
        results.append(None)


prompts_data['Response'] = results
prompts_data.to_csv('responses.csv', index=False)
prompts_data['Response'] = results
prompts_data.to_csv('responses.csv', index=False)


<ipython-input-31-d95a33ea98d5>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses_df['Response'] = results
<ipython-input-31-d95a33ea98d5>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses_df['Response'] = results
<ipython-input-31-d95a33ea98d5>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [ ]:
import json

cleaned_responses = []
for response in results:
    cleaned_response = response.replace('\n', '')
    cleaned_responses.append(cleaned_response)

cleaned_responses_json = [json.loads(response) for response in cleaned_responses]

In [ ]:
import pandas as pd
import json

def parse_json_response(response_str):
    try:
        corrected_str = response_str.replace('""', '"').strip('"')
        return json.loads(corrected_str)
    except json.JSONDecodeError:
        return None

def process_data_v2(file_path):
    data = pd.read_csv(file_path)
    data['R1_flag'] = False
    data['R2_flag'] = False
    data['is_academic'] = False
    data['country'] = None
    data['state'] = None

    for index, row in data.iterrows():
        response_json = parse_json_response(row['Response'])
        if response_json:
            countries = []
            states = []

            # Check if response contains multiple institutions
            institutions = response_json.get('institutions', [response_json]) \
                if isinstance(response_json, dict) else response_json

            for institution in institutions:
                country = institution.get('location', '')
                state = institution.get('country_or_state', '')
                if country:
                    countries.append(country)
                if state:
                    states.append(state)
                if 'R1' in institution.get('research_classification', ''):
                    data.at[index, 'R1_flag'] = True
                if 'R2' in institution.get('research_classification', ''):
                    data.at[index, 'R2_flag'] = True
                if institution.get('classification', '') == 'academic':
                    data.at[index, 'is_academic'] = True
            data.at[index, 'country'] = ', '.join(set(countries))
            data.at[index, 'state'] = ', '.join(set(states))

    return data

processed_data_v2 = process_data_v2(file_path)

processed_data_v2.head(30)
processed_data_v2.to_csv('processed_responses.csv', index=False)

In [ ]:
import pandas as pd
import json

def parse_json_response(response_str):
    corrected_str = response_str.replace('"{', '{').replace('}"', '}').replace('""', '"').strip('"')
    try:
        return json.loads(corrected_str)
    except json.JSONDecodeError:
        return None

def process_data(file_path):
    data = pd.read_csv(file_path)
    data['R1_flag'] = False
    data['R2_flag'] = False
    data['is_academic'] = False
    data['country'] = None
    data['state'] = None

    for index, row in data.iterrows():
        response_json = parse_json_response(row['Response'])
        if response_json:
            institutions = [response_json] if isinstance(response_json, dict) else response_json

            countries = set()
            states = set()
            for institution in institutions:
                country = institution.get('location', '')
                state = institution.get('country_or_state', '')
                if country:
                    countries.add(country)
                if state:
                    states.add(state)
                if 'R1' in institution.get('research_classification', ''):
                    data.at[index, 'R1_flag'] = True
                if 'R2' in institution.get('research_classification', ''):
                    data.at[index, 'R2_flag'] = True
                if institution.get('classification', '') == 'academic':
                    data.at[index, 'is_academic'] = True

            data.at[index, 'country'] = ', '.join(countries)
            data.at[index, 'state'] = ', '.join(states)

    return data

file_path = 'responses.csv'
processed_data = process_data(file_path)
processed_data.to_csv('processed_responses.csv', index=False)


In [ ]:
import pandas as pd

data = pd.read_csv('processed_responses.csv')

r1_count = data['R1_flag'].sum()
r2_count = data['R2_flag'].sum()

print(f"Number of R1 institutions: {r1_count}")
print(f"Number of R2 institutions: {r2_count}")


Number of R1 institutions: 941
Number of R2 institutions: 236


In [ ]:
import pandas as pd
import ast
from collections import Counter

data = pd.read_csv('processed_responses.csv')

data['UMLS_Codes'] = data['UMLS_Codes'].apply(lambda x: ast.literal_eval(x))
data['Keywords'] = data['Keywords'].apply(lambda x: ast.literal_eval(x))

umls_to_keyword = {}
for keywords, codes in zip(data['Keywords'], data['UMLS_Codes']):
    for keyword, code in zip(keywords, codes):
        if code not in umls_to_keyword:
            umls_to_keyword[code] = keyword.lower()

data['Mapped_Keywords'] = data['UMLS_Codes'].apply(lambda codes: [umls_to_keyword[code] for code in codes])
grouped_r1 = data[data['R1_flag']].groupby('Category')
grouped_r2 = data[data['R2_flag']].groupby('Category')

def calculate_keyword_frequencies(grouped_df):
    frequencies = {}
    for category, group in grouped_df:
        all_keywords = [keyword for sublist in group['Mapped_Keywords'] for keyword in sublist]
        frequencies[category] = pd.Series(all_keywords).value_counts()
    return frequencies

r1_keywords_freq = calculate_keyword_frequencies(grouped_r1)
r2_keywords_freq = calculate_keyword_frequencies(grouped_r2)


print("R1 Institutions:")
for category, freq in r1_keywords_freq.items():
    print(f"Category '{category}':")
    print(freq.head(10))
    print()

print("R2 Institutions:")
for category, freq in r2_keywords_freq.items():
    print(f"Category '{category}':")
    print(freq.head(10))
    print()



r1_df_list = [{'Category': category, 'Keyword': k, 'Frequency': v}
              for category, freqs in r1_keywords_freq.items()
              for k, v in freqs.head(10).items()]
r1_results_df = pd.DataFrame(r1_df_list)


r1_results_df.to_csv('r1_keyword_frequencies.csv', index=False)


r2_df_list = [{'Category': category, 'Keyword': k, 'Frequency': v}
              for category, freqs in r2_keywords_freq.items()
              for k, v in freqs.head(10).items()]
r2_results_df = pd.DataFrame(r2_df_list)


r2_results_df.to_csv('r2_keyword_frequencies.csv', index=False)


R1 Institutions:
Category 'asthma-pollution':
asthma treatment         53
lung diseases            14
air pollution            13
logistic regression       8
atopic dermatitis         8
comorbidities             8
allergic rhinitis         8
environmental factors     5
risk factors              5
health disparities        5
Name: count, dtype: int64

Category 'cardiovascular-with':
cardiovascular disease           189
heart failure                     31
risk factors                      30
social determinants of health     29
hypertension                      29
coronary heart disease            21
type 2 diabetes                   21
common diseases                   19
health disparities                18
genetic variants                  18
Name: count, dtype: int64

Category 'dementias-alzheimers':
american population                  106
adolescent symptoms of depression     39
hypertension                          14
risk factors                          14
logistic regression  

In [ ]:
import pandas as pd
import ast

# Load the data
file_path = 'processed_responses.csv'
data = pd.read_csv(file_path)

# Convert string representation of list in 'Institutions' to actual list
data['Institutions'] = data['Institutions'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Flatten the list of institutions to get a single list with all institutions
all_institutions = [institution for sublist in data['Institutions'] for institution in sublist]

# Calculate the number of unique institutions across all data
unique_institutions = set(all_institutions)
total_unique_institutions = len(unique_institutions)

# Group by 'Category' to find unique institutions for each category
category_institutions = data.explode('Institutions').groupby('Category')['Institutions'].nunique()

# Analyze multi-institutional teams and R2_flag
multi_institutional_data = data[data['Is Multi-Institutional'] == True]
multi_institutional_with_R2 = multi_institutional_data[multi_institutional_data['R2_flag'] == True ]
percentage_multi_institutional_with_R2 = (len(multi_institutional_with_R2) / len(multi_institutional_data) * 100
                                          if len(multi_institutional_data) > 0 else 0)

print(f"Total unique institutions: {total_unique_institutions}")
print(f"Unique institutions by category: {category_institutions.to_dict()}")
print(f"Percentage of multi-institutional teams involving at least one R2 institution: {percentage_multi_institutional_with_R2}%")


Total unique institutions: 283
Unique institutions by category: {'asthma-pollution': 61, 'cardiovascular-with': 153, 'dementias-alzheimers': 88, 'diabetes-with': 176, 'mental-health': 140}
Percentage of multi-institutional teams involving at least one R2 institution: 33.12883435582822%
